In [1]:
import cv2
import numpy as np
from matplotlib.colors import LogNorm
from scipy import signal
import utils
%matplotlib notebook
import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
from IPython.display import Image
from IPython.display import display

# Part1 hybrid image

#### read image

In [20]:
im1_file = './img1_lala.jpg'
im2_file = './2Women.jpg'

'''
im2_file = './rat.jpg'
im1_file = './cat2.jpg'
'''
'''
im1_file = './3cat.jpg'
im2_file = './3dog.jpg'
'''
im1_BGR = cv2.imread(im1_file)
im2_BGR = cv2.imread(im2_file)
im1_RGB = cv2.cvtColor(im1_BGR, cv2.COLOR_BGR2RGB)
im2_RGB = cv2.cvtColor(im2_BGR, cv2.COLOR_BGR2RGB)
im1 = cv2.cvtColor(im1_BGR, cv2.COLOR_BGR2GRAY)
im2 = cv2.cvtColor(im2_BGR, cv2.COLOR_BGR2GRAY)

#### plot fft function

In [21]:
def fft_image(img):
    fftmag = np.abs(np.fft.fftshift(np.fft.fft2(img)))
    plt.imshow(fftmag,norm=LogNorm(fftmag.min(),fftmag.max()),cmap='jet')

#### fft of input image

In [4]:
fig=plt.figure(figsize=(10, 10))
columns = 2
rows = 1
fig.add_subplot(rows, columns, 1)
fft_image(im1)
plt.title("cat")
fig.add_subplot(rows, columns, 2)
fft_image(im2)
plt.title("dog")
plt.show(block=True)

#### align image

In [22]:
pts_im1 = utils.prompt_eye_selection(im1)
plt.show(block=True)

In [23]:
pts_im2 = utils.prompt_eye_selection(im2)
plt.show(block=True)

In [24]:
im1, im2 = utils.align_images(im1_file, im2_file,pts_im1,pts_im2,save_images=False)

In [25]:
im1_RGB = cv2.cvtColor(im1, cv2.COLOR_BGR2RGB)
im2_RGB = cv2.cvtColor(im2, cv2.COLOR_BGR2RGB)

#### high and low pass filter function

In [26]:
def high_pass(img, dim, high_pass_fre):
    img_res = np.zeros(shape=img.shape, dtype=np.uint8)
    img_res[:,:,dim] = img[:,:,dim]
    
    img_res = img_res - cv2.GaussianBlur(img_res,(3*high_pass_fre,3*high_pass_fre), cv2.BORDER_DEFAULT)
    return img_res

def low_pass(img, low_pass_fre):
    img_res = img
    
    ksize = np.int(np.ceil(low_pass_fre)*3+1)
    fil = cv2.getGaussianKernel(ksize, low_pass_fre) # 1D kernel
    fil = fil*np.transpose(fil) # 2D kernel by outer product
    
    img_res = cv2.filter2D(img_res, -1, fil)
    return img_res

#### hybrid the image

In [27]:
high_pass_fre = 23
low_pass_fre = 7
# 23 7; 15 5
img2_grey = low_pass(im2, low_pass_fre)
img2 = cv2.cvtColor(img2_grey, cv2.COLOR_BGR2RGB)

img = np.zeros(shape=im1_RGB.shape, dtype=np.uint8)
img1 = np.zeros(shape=im1_RGB.shape, dtype=np.uint8)

for j in range(3):
    c_high = high_pass(im1_RGB, j, high_pass_fre)
    img1 += c_high


img = img1 + img2

plt.imshow(img)
plt.show(block=True)

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_20444\3041769544.py:11: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ksize = np.int(np.ceil(low_pass_fre)*3+1)


#### BW1 color hybrid

In [30]:
img2_grey = cv2.cvtColor(img2_grey, cv2.COLOR_BGR2GRAY)

In [31]:
img1_grey = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)

In [31]:
im_grays = img1_grey + img2_grey

In [32]:
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

In [33]:
for j in range(3):
    im_1col2gray = img1[...,j] + img2_grey
    im_1gray2col = img1_grey + img2[...,j]
    

In [34]:
plt.title("1col2gray")
plt.imshow(im_1col2gray)
plt.show(block=True)

In [69]:
plt.title("1gray2col")
plt.imshow(im_1gray2col, cmap='gray')
plt.show(block=True)

In [70]:
plt.title("both gray")
plt.imshow(im_grays, cmap='gray')
plt.show(block=True)

In [71]:
plt.title("both color")
plt.imshow(img_gray, cmap='gray')
plt.show(block=True)

#### plot frequency of filtered input image 

In [32]:
img1_grey = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
img2_grey = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)

In [33]:
fft_image(img1_grey)
plt.show(block=True)

In [34]:

fft_image(img2_grey)
plt.show(block=True)


#### plot filtered input image 

In [35]:
plt.imshow(img1)
plt.show(block=True)

In [37]:
plt.imshow(img2)
plt.show(block=True)

#### plot fft hybrid image 

In [200]:
fft_image(img)
plt.show(block=True)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


#### BW3 image pyramids

In [39]:
lower = img.copy()
 
# Implement a Gaussian Pyramid
gaussian_pyr = [lower]
for i in range(4):
    lower = cv2.pyrDown(lower)
    gaussian_pyr.append(lower)
 
#from last to first
laplacian_top = gaussian_pyr[-1]
 
# Implement Laplacian Pyramid
laplacian_pyr = [laplacian_top]
for i in range(4,0,-1):
    size = (gaussian_pyr[i - 1].shape[1], gaussian_pyr[i - 1].shape[0])
    gaussian_expanded = cv2.pyrUp(gaussian_pyr[i], dstsize=size)
    laplacian = cv2.subtract(gaussian_pyr[i-1], gaussian_expanded)
    laplacian_pyr.append(laplacian)

In [51]:
fig=plt.figure(figsize=(15, 15))
columns = 5
rows = 1
fig.add_subplot(rows, columns, 1)
plt.title("origin")
plt.imshow(gaussian_pyr[0])
fig.add_subplot(rows, columns, 2)
plt.imshow(gaussian_pyr[1])
plt.title("sub1")
fig.add_subplot(rows, columns, 3)
plt.imshow(gaussian_pyr[2])
plt.title("sub2")
fig.add_subplot(rows, columns, 4)
plt.imshow(gaussian_pyr[3])
plt.title("sub3")
fig.add_subplot(rows, columns, 5)
plt.imshow(gaussian_pyr[4])
plt.title("sub4")
plt.show(block=True)

In [52]:
fig=plt.figure(figsize=(15, 15))
columns = 5
rows = 1
fig.add_subplot(rows, columns, 1)
plt.title("origin")
plt.imshow(laplacian_pyr[4])
fig.add_subplot(rows, columns, 2)
plt.imshow(laplacian_pyr[3])
plt.title("sub1")
fig.add_subplot(rows, columns, 3)
plt.imshow(laplacian_pyr[2])
plt.title("sub2")
fig.add_subplot(rows, columns, 4)
plt.imshow(laplacian_pyr[1])
plt.title("sub3")
fig.add_subplot(rows, columns, 5)
plt.imshow(laplacian_pyr[0])
plt.title("sub4")
plt.show(block=True)

# Part2

## Contrast Enhancement

In [3]:
from skimage import exposure

In [4]:
im_file = './poor_contrast.jpg'

im_BGR = cv2.imread(im_file)
im_RGB = cv2.cvtColor(im_BGR,cv2.COLOR_BGR2RGB)
plt.imshow(im_RGB)
plt.show(block=True)

In [5]:
def apply_CLAHE_channel(img, dim):
    channel = np.zeros(shape=img.shape, dtype=np.uint8)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(12,12))
    channel[:,:,dim] = clahe.apply(img[:,:,dim])
    img[:,:,dim] = channel[:,:,dim]

In [6]:
im_HSV = cv2.cvtColor(im_RGB, cv2.COLOR_RGB2HSV)
apply_CLAHE_channel(im_HSV, 2)
plt.imshow(cv2.cvtColor(im_HSV, cv2.COLOR_HSV2RGB))
plt.show(block=True)

## Color enhancement

In [12]:
im1_file = './poor_contrast.jpg'
im1 = cv2.imread(im1_file)
im1_orig = cv2.cvtColor(im1,cv2.COLOR_BGR2RGB)

hsv = cv2.cvtColor(im1,cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(hsv)
'''
value = 50
lim = 255 - value
v[v > lim] = 255
v[v <= lim] += value

'''
s_brighter = exposure.adjust_gamma(s, 0.5)
final_hsv = cv2.merge((h, s_brighter, v))
im1_out = cv2.cvtColor(final_hsv,cv2.COLOR_HSV2RGB)
plt.figure()
plt.imshow(im1_out)
plt.show(block = True)

## Color Shift BW2

In [106]:
im_file_cs = './img1_lala.jpg'
im_cs = cv2.imread(im_file_cs)
im_cs_LAB = cv2.cvtColor(im_cs, cv2.COLOR_BGR2Lab)
l, a, b = cv2.split(im_cs_LAB)

In [110]:
#more red
a_more_red = exposure.adjust_gamma(a, 0.8)
b_less_yellow = exposure.adjust_gamma(b, 1.2)
im_cs_red = cv2.merge((l, a_more_red, b))
im_cs_yellow = cv2.merge((l, a, b_less_yellow))

In [111]:
im_cs_red = cv2.cvtColor(im_cs_red, cv2.COLOR_Lab2RGB)
im_cs_yellow = cv2.cvtColor(im_cs_yellow, cv2.COLOR_Lab2RGB)

In [112]:
plt.imshow(im_cs_red)
plt.show(block=True)

In [113]:
plt.imshow(im_cs_yellow)
plt.show(block=True)